In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# # 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

# landsat8

In [ ]:
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# ,LANDSAT/LC08/C01/T1_SR/LC08_123039_20200803
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_118028_20200613')
Map.addLayer(image,visParams,'image')
Map.centerObject(image,7)
print(image.get('SOLAR_AZIMUTH_ANGLE').getInfo())
print(image.get('SOLAR_ZENITH_ANGLE').getInfo())

In [ ]:
a = image.clip(roi)
print(a.geometry().getInfo())

In [ ]:
polyson = ee.Geometry.Polygon([[[125.30550000000001, 46.46529007795952], [125.22422552962367, 46.2338567558239], [125.08639241302929, 45.83729986498], [125.30550000000001, 45.8362], [125.30550000000001, 46.46529007795952]]])
Map.addLayer(polyson, {}, "polyson")

In [ ]:
a = str(image.select('B2').getInfo()['bands'][0]['crs'])
print(a)

# sentinel2

In [ ]:
# 20200427T030539_20200427T031145_T49REQ
# 20200427T030539_20200427T031145_T49REP
# 20200427T030539_20200427T031145_T49REN
# sentinel-2数据

def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

s2 = ee.Image('COPERNICUS/S2_SR/20200427T030539_20200427T031145_T49REP')
Map.addLayer(s2.clip(bound), {'bands': ['B8', 'B11', 'B4'], 'min':0,'max': 3000}, 'S2')

# 去阴影

In [ ]:
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi).reproject('EPSG:3857',None,30)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

In [ ]:
mask_image = maskSR(image)
Map.addLayer(mask_image,visParams,'mask_image')

In [ ]:
azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
zenith = image.get('SOLAR_ZENITH_ANGLE')
DEM_Mercator = ee.Image('USGS/SRTMGL1_003').select('elevation').reproject('EPSG:3857',None,30)
Shadow_pro=ee.Terrain.hillShadow(DEM_Mercator,azimuth,zenith,200,True)
# Map.addLayer(Shadow_pro.remap([0,1],[1,0]).selfMask(),{'palette':['red']},'Shadow_pro')
Map.addLayer(image.updateMask(Shadow_pro),visParams,'Shadow_pro200')

In [ ]:
azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
zenith = image.get('SOLAR_ZENITH_ANGLE')
hillshade = ee.Terrain.hillshade(ee.Image('USGS/SRTMGL1_003').select('elevation').clip(image.geometry()),azimuth,zenith)
Map.addLayer(hillshade,{},'hillshade')

shade = hillshade.eq(0)
Map.addLayer(shade.selfMask(),{'palette':['blue']},'shade')

In [ ]:
# ee.Algorithms.HillShadow
DEM_Mercator_nopro = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(image.geometry()).reproject('EPSG:3857',None,30)
Shadow_nopro = ee.Algorithms.HillShadow(DEM_Mercator_nopro,azimuth,zenith,200)
Map.addLayer(Shadow_nopro.remap([0,1],[1,0]).selfMask(),{'palette':['red']},'Shadow_1')
# Map.addLayer(image.updateMask(Shadow_nopro),visParams,'Shadow_nopro')

In [ ]:
# 最终版
DEM_Mercator_nopro = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(image.geometry()).reproject('EPSG:3857',None,30)
Shadow_nopro = ee.Terrain.hillShadow(DEM_Mercator_nopro,azimuth,zenith,200,True)
Map.addLayer(Shadow_nopro.remap([0,1],[1,0]).selfMask(),{'palette':['red']},'Shadow_1')